In [140]:
import numpy as np
import pandas as pd
import re
from itertools import product

In [143]:
n_prod = 5
n_inj = 3
n_wells = n_prod + n_inj

n_runs = 15
nx, ny, nz = 108, 100, 63
nt = nx * ny * nz

n_tsteps = 20    # Enter the number of Dates exported in Eclipse Datafile

In [144]:
# function for reading the text files and extracting the numbers
def splitter(file):

    lines = file.readlines()
    B = [x for x in lines if not x.startswith('--')]
    C = " ".join(B)
    nums = []
    #                            3*4.52          4*0       0\n    24*0.00       4.235e-10 or 4.235E-10          5e-10 or 5E-10 
    for n in re.findall(r'\d+\*\d*\.\d+|\d+\*[-+]\d+\.\d+|\d+\*\d+|[0]\|[0]\n|\d+\.\d+(?:e[+-]\d+|E[-+]\d+)?|\d+(?:e[+-]\d+|E[-+]\d+)?', C):
        split_by_ast = n.split("*")
        if len(split_by_ast) == 1:
            nums += [float(split_by_ast[0])]
        else:
            nums += [float(split_by_ast[1])] * int(split_by_ast[0])
    return nums


def repeat(x):
    A = list(np.array(x)[actives])*n_runs
    B = np.array(A)
    return B

In [145]:
a = open('C:/Users/pymnb/Downloads/VOLVE/15/features/001.soil.for.location.txt', 'r')

SOIL = np.array(splitter(a)).reshape((nt, 1), order='F')
SOIL[SOIL < 0.053] = 0

#removing a few grids where they were out of the main reservoir in VOLVE field
indices = [554377, 565177, 575977, 576085, 586777, 586885, 597577, 597685, 608377, 608485, 619177,\
619285, 629977, 630085, 640777, 640885, 651577, 651685, 662377, 662485, 673177, 673285]

SOIL[indices]=0
actives = np.where(SOIL!=0)[0]

# Create the ijk_r
T1 = sorted(list(product(range(1,nx+1), range(1,ny+1), range(1,nz+1))), key=lambda x: (x[2], x[1], x[0]))
ijk_all = [list(x) for x in T1]
ijk = repeat(ijk_all)

### injectors and producers wellpathes through grids

In [149]:
# For injectors
a2 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/005.IF1B_path.txt', 'r').readlines())
a3 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/005.IF4_path.txt', 'r').readlines())
a4 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/005.IF5_path.txt', 'r').readlines())

a2, a3, a4 = (re.findall(r'\d+', a2)) , (re.findall(r'\d+', a3)), (re.findall(r'\d+', a4))  # it generates a list of strings
a2, a3, a4 = list(map(int, a2)), list(map(int, a3)), list(map(int, a4))     # it generates a list of numbers
inj_path_IF1B = np.array(a2).reshape((58,3), order = 'C')
inj_path_IF4 = np.array(a3).reshape((62,3), order = 'C')
inj_path_IF5 = np.array(a4).reshape((63,3), order = 'C')
inj_path = np.concatenate([inj_path_IF1B, inj_path_IF4, inj_path_IF5], axis=0)

# For producers
b2 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/006.PF1C_path.txt', 'r').readlines())
b3 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/006.PF11B_path.txt', 'r').readlines())
b4 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/006.PF12_path.txt', 'r').readlines())
b5 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/006.PF14_path.txt', 'r').readlines())
b6 = " ".join(open('C:/Users/pymnb/Downloads/VOLVE/15/features/006.PF15C_path.txt', 'r').readlines())

b2, b3, b4, b5, b6 = (re.findall(r'\d+', b2)), (re.findall(r'\d+', b3)), (re.findall(r'\d+', b4)),\
                     (re.findall(r'\d+', b5)), (re.findall(r'\d+', b6))
b2, b3, b4, b5, b6 = list(map(int, b2)), list(map(int, b3)), list(map(int, b4)), list(map(int, b5)), list(map(int, b6))
prod_path_PF1C = np.array(b2).reshape((103,3), order = 'C')
prod_path_PF11B = np.array(b3).reshape((91,3), order = 'C')
prod_path_PF12 = np.array(b4).reshape((52,3), order = 'C')
prod_path_PF14 = np.array(b5).reshape((112,3), order = 'C')
prod_path_PF15C = np.array(b6).reshape((49,3), order = 'C')
prod_path = np.concatenate([prod_path_PF1C, prod_path_PF11B, prod_path_PF12, prod_path_PF14, prod_path_PF15C], axis=0)

df_path_PF1C, df_path_PF11B, df_path_PF12, df_path_PF14, df_path_PF15C = pd.DataFrame(prod_path_PF1C),\
                                                        pd.DataFrame(prod_path_PF11B), pd.DataFrame(prod_path_PF12),\
                                                        pd.DataFrame(prod_path_PF14), pd.DataFrame(prod_path_PF15C)
df_path_IF1B, df_path_IF4, df_path_IF5 = pd.DataFrame(inj_path_IF1B), pd.DataFrame(inj_path_IF4), pd.DataFrame(inj_path_IF5)

### Find the ijk of surrounding grids of well paths

In [150]:
# This function get the ijk of well path, and the number of surrounding tiers and returns the ijk of surrounding

def well_surrounding(x, nlayer_sur):
    
    G = x
    for p in range(1,nlayer_sur+1):
        i_low, i_up = x[0]-p, x[0]+p
        j_low, j_up = x[1]-p, x[1]+p
        k_low, k_up = x[2]-p, x[2]+p
        # grids in (i+1) or (j-1) or (j+1) or (j-1)
        I1 = pd.concat([i_low, x.iloc[:,1:3]], axis=1)
        I2 = pd.concat([i_up,  x.iloc[:,1:3]], axis=1)
        J1 = pd.concat([x.iloc[:,0], j_low, x.iloc[:,2]], axis=1)
        J2 = pd.concat([x.iloc[:,0], j_up, x.iloc[:,2]], axis=1)
        # grids in (k+1) or (k-1)
        K1 = pd.concat([x.iloc[:,0:2], k_low], axis=1)
        K2 = pd.concat([x.iloc[:,0:2], k_up], axis=1)
        # grids in horizontal corners
        IJ1 = pd.concat([i_low, j_low, x.iloc[:,2]], axis=1)
        IJ2 = pd.concat([i_low, j_up, x.iloc[:,2]], axis=1)
        IJ3 = pd.concat([i_up, j_up, x.iloc[:,2]], axis=1)
        IJ4 = pd.concat([i_up, j_low, x.iloc[:,2]], axis=1)
        # grids in vertical corners
        JK1 = pd.concat([x.iloc[:,0], j_up, k_up], axis=1)
        JK2 = pd.concat([x.iloc[:,0], j_low, k_up], axis=1)
        JK3 = pd.concat([x.iloc[:,0], j_up, k_low], axis=1)
        JK4 = pd.concat([x.iloc[:,0], j_low, k_low], axis=1)
        # grids in vertical corners
        IK1 = pd.concat([i_up, x.iloc[:,1], k_up], axis=1)
        IK2 = pd.concat([i_low, x.iloc[:,1], k_up], axis=1)
        IK3 = pd.concat([i_up, x.iloc[:,1], k_low], axis=1)
        IK4 = pd.concat([i_low, x.iloc[:,1], k_low], axis=1)
        
        N = pd.concat([I1, I2, J1, J2, IJ1, IJ2, IJ3, IJ4, JK1, JK2, JK3, JK4, IK1, IK2, IK3, IK4, K1, K2])
        dfnew = pd.concat([G, N])
        G = dfnew
            
    F = G.drop_duplicates(keep="last")
    ijk_surrounding = F[F.min(axis=1) > 0].reset_index(drop=True)

    #get the indexes of sourrounding grids in the main dataset
    index_surrounding = ijk_surrounding.merge(pd.DataFrame(ijk).drop_duplicates().reset_index())['index']
    index_surrounding = index_surrounding.to_frame()
    #index_surrounding = index_surrounding.sort_values(0)
    return index_surrounding

ijk_surrounding = well_surrounding(df_path_PF1C, 5)

# This part will be used in the prediction files 5
well_names = [df_path_PF1C, df_path_PF11B, df_path_PF12, df_path_PF14, df_path_PF15C, df_path_IF1B, df_path_IF4, df_path_IF5]
J =pd.DataFrame()
for i in well_names:
    for j in range(1,5+1):
        A = well_surrounding(i, j)
        B = pd.concat([J, A], axis=1)
        J = B
#J.to_csv('C:/Users/pymnb/Downloads/VOLVE/15/features/index_tiers.csv', index = False, header=False)

### Load the initial design

In [152]:
df_lhs = pd.read_csv('C:/Users/pymnb/Downloads/VOLVE/15/LHS_design.csv')

df_lhs_rep = list(np.repeat(df_lhs.values, n_tsteps, axis=0))*n_prod
df_lhs_rep = pd.DataFrame(df_lhs_rep)
df_lhs_rep.columns = ["pf11b_rates", "pf12_rates", "pf14_rates", "pf1c_rates", "pf15c_rates"]

,pf11b_rates,pf12_rates,pf14_rates,pf1c_rates,pf15c_rates
0,2632,1521,4027,68,3665
1,2632,1521,4027,68,3665
2,2632,1521,4027,68,3665
3,2632,1521,4027,68,3665
4,2632,1521,4027,68,3665
...,...,...,...,...,...
1495,2815,2091,2458,2653,2972
1496,2815,2091,2458,2653,2972
1497,2815,2091,2458,2653,2972
1498,2815,2091,2458,2653,2972


# Importing the Original well Dynamic parameters

In [153]:
prod = open('C:/Users/pymnb/Downloads/VOLVE/15/features/032.productions.txt', 'r')

prod_expanded = np.array(splitter(prod)).reshape(n_tsteps*n_runs, n_prod*8)  # 9: number of dynamic features for producers
 
df_prod_orig = pd.DataFrame(prod_expanded)
df_prod_orig.columns = ["OPR(PF14)", "OPR(PF12)", "OPR(PF15C)", "OPR(PF11B)", "OPR(PF1C)",\
                        "WPR(PF14)", "WPR(PF12)", "WPR(PF15C)", "WPR(PF11B)", "WPR(PF1C)",\
                        "WCT(PF14)", "WCT(PF12)", "WCT(PF15C)", "WCT(PF11B)", "WCT(PF1C)",\
                        "OPT(PF14)", "OPT(PF12)", "OPT(PF15C)", "OPT(PF11B)", "OPT(PF1C)",\
                        "WPT(PF14)", "WPT(PF12)", "WPT(PF15C)", "WPT(PF11B)", "WPT(PF1C)",\
                        "GOR(PF14)", "GOR(PF12)", "GOR(PF15C)", "GOR(PF11B)", "GOR(PF1C)",\
                        "GPR(PF14)", "GPR(PF12)", "GPR(PF15C)", "GPR(PF11B)", "GPR(PF1C)",\
                        "BHP(PF14)", "BHP(PF12)", "BHP(PF15C)", "BHP(PF11B)", "BHP(PF1C)"]

In [154]:
ini_1 = pd.DataFrame()

for i in range(0,8):
    AA1 = pd.concat([df_prod_orig.iloc[:,i*n_prod], df_prod_orig.iloc[:,i*n_prod+1], df_prod_orig.iloc[:,i*n_prod+2],\
                     df_prod_orig.iloc[:,i*n_prod+3], df_prod_orig.iloc[:,i*n_prod+4]]).reset_index(drop=True)
    BB1 = pd.concat([ini_1, AA1], axis=1)
    ini_1 = BB1
    
df_prod = ini_1
df_prod.columns = ["OPR", "WPR", "WCT", "OPT", "WPT", "GOR", "GPR", "BHP"]

,OPR,WPR,WCT,OPT,WPT,GOR,GPR,BHP
0,329.39910,2302.601,0.874848,4167373.0,7245203.0,134.2442,44219.910,330.0313
1,301.89140,2330.109,0.885300,4225711.0,7671154.0,133.9747,40445.800,332.9027
2,268.39320,2363.607,0.898027,4277247.0,8096010.0,133.6536,35871.730,334.0029
3,239.46780,2392.532,0.909017,4323788.0,8533756.0,133.3226,31926.460,334.6493
4,217.42970,2414.570,0.917390,4364989.0,8968947.0,133.0540,28929.900,335.1245
...,...,...,...,...,...,...,...,...
1495,38.63411,2330.888,0.983695,719251.0,6706055.0,148.4855,5736.604,367.5684
1496,36.35754,2332.504,0.984652,726029.2,7128092.0,148.4748,5398.178,367.7827
1497,34.06202,2334.734,0.985621,732499.3,7557462.0,148.4607,5056.869,367.9739
1498,32.08974,2336.491,0.986452,738476.2,7980211.0,148.4485,4763.673,368.1506


# Create the column for Average characteristics of Tiers

#### Open the dynamic parameters for all grids

In [124]:
so = open('C:/Users/pymnb/Downloads/VOLVE/15/features/030.SO.txt', 'r')
p  = open('C:/Users/pymnb/Downloads/VOLVE/15/features/031.P.txt', 'r')
poro = open('C:/Users/pymnb/Downloads/VOLVE/15/features/024.poro.txt', 'r')
permx = open('C:/Users/pymnb/Downloads/VOLVE/15/features/021.permx.txt', 'r')
permy = open('C:/Users/pymnb/Downloads/VOLVE/15/features/022.permy.txt', 'r')
permz = open('C:/Users/pymnb/Downloads/VOLVE/15/features/023.permz.txt', 'r')

so = np.array(splitter(so)).reshape((nx*ny*nz, 40, n_runs), order="F")
p  = np.array(splitter(p)).reshape((nx*ny*nz, 40, n_runs), order="F")
poro = np.array(splitter(poro)).reshape((nx*ny*nz, 1, 1), order="F")
permx = np.array(splitter(permx)).reshape((nx*ny*nz, 1, 1), order="F")
permy = np.array(splitter(permy)).reshape((nx*ny*nz, 1, 1), order="F")
permz = np.array(splitter(permz)).reshape((nx*ny*nz, 1, 1), order="F")

#### function for averaging a dynamic variable for selected grids in surrounding of wells

In [155]:
def AVERAGE(dynamic_feature ,name, timestep_no, run_no, tier_no):
    
    # get a slice of the dynamic paramter for one timestep
    var_slice_timestep = dynamic_feature[actives][:,(timestep_no-1),:]
    
    # get a slice of dynamic variable for one run
    var_slice_run = var_slice_timestep[:,(run_no-1)]
    
    # get the dynamic variable for grids in surrounding, here the well_surrounding() function is called from before
    var_slice_time_surr = var_slice_run[well_surrounding(name, tier_no)]
    
    # calculate the average that is for the selected timestep ans run
    average = np.reshape(np.average(var_slice_time_surr), (1,1))
    
    return average

# so, sw, p
# df_path_PF1C, df_path_PF11B, df_path_PF12, df_path_PF14, df_path_PF15C
# 1 < timestep_no < 40   and 1 < run_no < 15
AA = AVERAGE(p, df_path_PF15C, 1, 1, 4)
print(AA)

[[373.759433]]


#### Create average charactersitics columns (takes time to run)

In [126]:
from timeit import default_timer as timer
start = timer()


z3, z4= np.zeros((1500,1)), np.zeros((1500,1))

well_names = [df_path_PF14, df_path_PF12, df_path_PF15C, df_path_PF11B, df_path_PF1C]

for m in range(1,5+1):
    z1, z2 = np.zeros((1,1)), np.zeros((1,1))
    for i in well_names:
        for j in range(1, n_runs+1):
            for k in range (1, n_tsteps+1):
                A1, A2 = AVERAGE(so, i, k, j, m), AVERAGE(p, i, k, j, m)
                B1, B2 = np.concatenate([z1, A1]), np.concatenate([z2, A2])
                z1, z2 = B1, B2    
    SO_av, P_av = B1[1:], B2[1:]
    C1, C2 = np.concatenate([z3, SO_av], axis=1), np.concatenate([z4, P_av], axis=1)
    z3, z4 = C1, C2

SO_T,  P_T = C1[:,1:], C2[:,1:]
SO_T,  P_T = pd.DataFrame(SO_T), pd.DataFrame(P_T)

df_average_dynamic = pd.concat([SO_T, P_T], axis=1)
df_average_dynamic.columns = ["SO_T1", "SO_T2", "SO_T3", "SO_T4", "SO_T5",\
                              "P_T1" , "P_T2" , "P_T3" , "P_T4" , "P_T5"] 

end = timer()
print("Run Time: "+str((end - start)//60)+" min")

Run Time: 110.0 min


In [136]:
z5, z6, z7, z8 = np.zeros((5,1)), np.zeros((5,1)), np.zeros((5,1)), np.zeros((5,1))

well_names = [df_path_PF14, df_path_PF12, df_path_PF15C, df_path_PF11B, df_path_PF1C]

for m in range(1,5+1):
    z9, z10, z11, z12 = np.zeros((1,1)), np.zeros((1,1)), np.zeros((1,1)), np.zeros((1,1))
    for i in well_names:
        A5, A6, A7, A8 = AVERAGE(poro, i, 1, 1, m), AVERAGE(permx, i, 1, 1, m), AVERAGE(permy, i, 1, 1, m), AVERAGE(permz, i, 1, 1, m)
        B5, B6, B7, B8 = np.concatenate([z9, A5]), np.concatenate([z10, A6]), np.concatenate([z11, A7]), np.concatenate([z12, A8])
        z9, z10, z11, z12 = B5, B6, B7, B8    
    poro_av, permx_av, permy_av, permz_av = B5[1:], B6[1:], B7[1:], B8[1:]
    C5, C6, C7, C8 = np.concatenate([z5, poro_av], axis=1), np.concatenate([z6, permx_av], axis=1),\
                    np.concatenate([z7, permy_av], axis=1), np.concatenate([z8, permz_av], axis=1)
    z5, z6, z7, z8 = C5, C6, C7, C8

poro_T,  permx_T, permy_T, permz_T = C5[:,1:], C6[:,1:], C7[:,1:], C8[:,1:]
poro_T,  permx_T, permy_T, permz_T = pd.DataFrame(poro_T), pd.DataFrame(permx_T), pd.DataFrame(permy_T), pd.DataFrame(permz_T)

df_average_static = pd.concat([poro_T,  permx_T, permy_T, permz_T], axis=1)

df_average_static = pd.DataFrame(np.repeat(df_average_static.values, 600, axis=0))
df_average_static.columns = ["poro_T1",   "poro_T2",   "poro_T3",   "poro_T4",   "poro_T5",\
                             "permx_T1" , "permx_T2" , "permx_T3" , "permx_T4" , "permx_T5",\
                             "permy_T1" , "permy_T2" , "permy_T3" , "permy_T4" , "permy_T5",\
                             "permz_T1" , "permz_T2" , "permz_T3" , "permz_T4" , "permz_T5"] 

### Creat Indexis Columns for timesteps, runs, wells

In [128]:
time_ind  = np.array([*range(1,n_tsteps+1)]*(n_prod*n_runs)).reshape(n_prod*n_tsteps*n_runs,1)
time_ind = pd.DataFrame(time_ind)

well_ind = np.repeat(np.arange(1,n_prod+1),(n_runs*n_tsteps)).reshape(n_prod*n_tsteps*n_runs,1)
well_ind = pd.DataFrame(well_ind)

df_ind = pd.concat([time_ind, well_ind], axis=1)
df_ind.columns = ["time_ind", "well_ind"]

### Prepare the distances of wells from producers and injectors at different intervals

In [134]:
# load the distance DataFrame
df_distance = pd.read_csv('C:/Users/pymnb/Downloads/VOLVE/15/features/distances.csv')
display(df_distance)

,1st prod-p1,2nd prod-p1,3rd prod-p1,1st inj-p1,2nd inj-p1,3rd inj-p1,1st prod-p2,2nd prod-p2,3rd prod-p2,1st inj-p2,...,3rd prod-p3,1st inj-p3,2nd inj-p3,3rd inj-p3,1st prod-p4,2nd prod-p4,3rd prod-p4,1st inj-p4,2nd inj-p4,3rd inj-p4
0,280.313495,405.309438,653.541964,517.008656,981.605182,991.972813,458.811035,521.954221,771.041365,434.737001,...,912.978540,546.152706,665.055749,1093.208260,697.362467,727.776182,1040.118652,492.856934,658.592961,1190.064753
1,280.313495,405.309438,653.541964,517.008656,981.605182,991.972813,458.811035,521.954221,771.041365,434.737001,...,912.978540,546.152706,665.055749,1093.208260,697.362467,727.776182,1040.118652,492.856934,658.592961,1190.064753
2,280.313495,405.309438,653.541964,517.008656,981.605182,991.972813,458.811035,521.954221,771.041365,434.737001,...,912.978540,546.152706,665.055749,1093.208260,697.362467,727.776182,1040.118652,492.856934,658.592961,1190.064753
3,280.313495,405.309438,653.541964,517.008656,981.605182,991.972813,458.811035,521.954221,771.041365,434.737001,...,912.978540,546.152706,665.055749,1093.208260,697.362467,727.776182,1040.118652,492.856934,658.592961,1190.064753
4,280.313495,405.309438,653.541964,517.008656,981.605182,991.972813,458.811035,521.954221,771.041365,434.737001,...,912.978540,546.152706,665.055749,1093.208260,697.362467,727.776182,1040.118652,492.856934,658.592961,1190.064753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,405.309438,585.807064,694.058147,552.424362,1282.240048,1366.804639,458.811035,821.222343,901.165817,346.265287,...,1260.092352,188.460990,1264.307913,1504.574793,727.776182,1142.324167,1419.805410,76.889109,1206.200844,1674.589878
2996,405.309438,585.807064,694.058147,552.424362,1282.240048,1366.804639,458.811035,821.222343,901.165817,346.265287,...,1260.092352,188.460990,1264.307913,1504.574793,727.776182,1142.324167,1419.805410,76.889109,1206.200844,1674.589878
2997,405.309438,585.807064,694.058147,552.424362,1282.240048,1366.804639,458.811035,821.222343,901.165817,346.265287,...,1260.092352,188.460990,1264.307913,1504.574793,727.776182,1142.324167,1419.805410,76.889109,1206.200844,1674.589878
2998,405.309438,585.807064,694.058147,552.424362,1282.240048,1366.804639,458.811035,821.222343,901.165817,346.265287,...,1260.092352,188.460990,1264.307913,1504.574793,727.776182,1142.324167,1419.805410,76.889109,1206.200844,1674.589878


### Create the final dataset

In [135]:
df = pd.concat([df_ind, df_lhs_rep, df_distance, df_average_static, df_average_dynamic, df_prod], axis=1).reset_index(drop=True) 
display(df)
df.to_csv('C:/Users/pymnb/Downloads/VOLVE/15/Datasets/Well-Non-Normalized.csv', index = False)

,time_ind,well_ind,pf11b_rates,pf12_rates,pf14_rates,pf1c_rates,pf15c_rates,1st prod-p1,2nd prod-p1,3rd prod-p1,...,P_T4,P_T5,OPR,WPR,WCT,OPT,WPT,GOR,GPR,BHP
0,1,1,2632,1521,4027,68,3665,280.313495,405.309438,653.541964,...,381.749851,381.715753,320.58930,2311.411,0.878196,4137531.0,7035664.0,134.8147,43220.170,326.8993
1,2,1,2632,1521,4027,68,3665,280.313495,405.309438,653.541964,...,384.424678,384.453292,329.38900,2302.611,0.874852,4167390.0,7245318.0,134.2466,44219.360,330.0333
2,3,1,2632,1521,4027,68,3665,280.313495,405.309438,653.541964,...,386.107565,386.162571,317.91100,2314.089,0.879213,4197202.0,7457650.0,134.1455,42646.330,331.8988
3,4,1,2632,1521,4027,68,3665,280.313495,405.309438,653.541964,...,387.102282,387.172595,301.47320,2330.527,0.885459,4225726.0,7671270.0,133.9739,40389.530,332.9036
4,5,1,2632,1521,4027,68,3665,280.313495,405.309438,653.541964,...,387.746065,387.830615,284.50980,2347.490,0.891904,4252119.0,7881757.0,133.8133,38071.210,333.5474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,36,5,2815,2091,2458,2653,2972,405.309438,585.807064,694.058147,...,382.841223,382.888133,34.00418,2334.679,0.985644,732501.0,7557578.0,148.4603,5048.270,367.9740
2996,37,5,2815,2091,2458,2653,2972,405.309438,585.807064,694.058147,...,382.892707,382.938340,33.00297,2335.659,0.986067,735517.3,7767746.0,148.4540,4899.422,368.0633
2997,38,5,2815,2091,2458,2653,2972,405.309438,585.807064,694.058147,...,382.944063,382.988634,32.03669,2336.417,0.986474,738477.8,7980327.0,148.4482,4755.787,368.1506
2998,39,5,2815,2091,2458,2653,2972,405.309438,585.807064,694.058147,...,382.993904,383.037276,31.13573,2337.111,0.986853,741385.2,8195311.0,148.4429,4621.876,368.2375
